# IMDb data
Number of movies that featured an actor/actress. IMDb publishes their data on https://www.imdb.com/interfaces/, so no need for webscraping. I used the `imdb-sqlite` package to parse IMDb's data into an SQlite database, which can be queried easily using `sqlalchemy`. 

In [ ]:
from sqlalchemy import inspect
from sqlalchemy import create_engine

engine = create_engine('sqlite:///imdb.db')
engine.connect()

inspector = inspect(engine)
table_names = inspector.get_table_names()

for table_name in table_names:
    print(f"Table:{table_name}")
    column_items = inspector.get_columns(table_name)
    print('\t'.join(n for n in column_items[0]))
    for c in column_items:
        assert len(c) == len(column_items[0])
        print('\t'.join(str(c[n]) for n in c))
    print()

In [ ]:
# Setting up the classes SQLalchemy needs to work with the SQlite database

from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, String, Integer

Base = declarative_base()

class People(Base):
    __tablename__ = "people"
    
    person_id = Column(String, primary_key=True)
    name = Column(String)
    born = Column(Integer)
    died = Column(Integer)

class Titles(Base):
    __tablename__ = 'titles'
    
    title_id = Column(String, primary_key=True)
    type = Column(String)
    primary_title = Column(String)
    original_title = Column(String)
    is_adult = Column(Integer)
    premiered = Column(Integer)
    ended = Column(Integer)
    runtime_minutes = Column(Integer)
    genres = Column(String)
    

class Crew(Base):
    __tablename__ = 'crew'
    
    title_id = Column(String, primary_key=True)
    person_id = Column(String)
    category = Column(String)
    job = Column(String)
    characters = Column(String)
    
    def __repr__(self):
        return f"{self.title_id} - {self.person_id} - {self.category} - {self.job} - {self.characters}"

In [ ]:
from tqdm import tqdm
from collections import Counter
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

actors = [
    "Brad Pitt",
    "Angelina Jolie",
    "Johnny Depp",
    "Nicolas Cage",
    "Morgan Freeman",
    "Samuel L. Jackson",
    "Benedict Cumberbatch",
    "Robert Downey Jr.",
    "Ryan Gosling",
    "Liam Neeson",
    "Will Smith",
    "Leonardo DiCaprio",
    "Carice van Houten",
    "Hugh Jackman",
    "Tom Cruise",
    "Scarlett Johansson",
    "Will Smith",
    "Clint Eastwood",
    "Chris Pratt",
    "Jennifer Aniston",
    "Julia Roberts",
    "Anna Kendrick",
    "Kate Winslet"
]

per_actor = {}

for actor in tqdm(actors):
    result = session.query(Crew, Titles.primary_title, Titles.premiered, People.name).join(Titles, Titles.title_id==Crew.title_id).join(People, People.person_id==Crew.person_id).filter(People.name==actor, Titles.type=="movie").all()
    years = [title.premiered for title in result]
    per_actor[actor] = {
        "movies_per_year": dict(Counter(years)),
        "total": len(years)
    }

In [ ]:
years = list(range(2014, 2021))
years

In [ ]:
imdb_time_series = []

for actor in per_actor:
    imdb_time_series.append({
        "title": {
            "english": f"Number of movies featuring {actor}",
            "dutch": f"Aantal films met {actor}"
        },
        "axis_label": {
            "english": "Number of movies",
            "dutch": "Aantal films"
        },
        "source": "https://www.imdb.com/interfaces/",
        "values": [per_actor[actor]['movies_per_year'].get(year, 0) for year in years]
    }) 
    
imdb_time_series

# KNMI data
Annual reviews of the KNMI (Dutch meteorology institute), counting the number of days in certain categories. The tables on their website can be scraped effortlessly straight into a Pandas dataframe using read_html()-functionality, which works great for this website and in one line of code returns a DataFrame containing the data we wanted. 

In [ ]:
import pandas as pd

days = {}

for year in years:
    table = pd.read_html(f'https://www.knmi.nl/nederland-nu/klimatologie/maand-en-seizoensoverzichten/{year}/jaar')
    
    table = table[0].replace('zomers dagen', 'zomerse dagen')
    
    try:
        days[year] = dict(zip(table[2][1:], table[0][1:]))
    except KeyError:
        days[year] = dict(zip(table.iloc[:, 2][1:], table.iloc[:, 0][1:]))


In [ ]:
types_of_day = {
    'Tropische dagen': 'tropical days (>25 degrees Celcius)',
    'Zomerse dagen': 'summer days (>25 degrees Celcius)',
    'Warme dagen': 'warm days (>20 degrees Celcius)',
    'Vorstdagen': 'freezing days (<0 degrees Celcius)'
}

climate_time_series = []
for type_of_day in types_of_day.keys():
    values = []
    for year in years:
        if type_of_day in days[year]:
            values.append(int(days[year][type_of_day]))
        else:
            values.append(int(days[year][type_of_day.lower()]))
            
    climate_time_series.append({
        "title": {
            "english": f"Number of {types_of_day[type_of_day]} in de Bilt",
            "dutch": f"Aantal {type_of_day.lower()} in de Bilt"
        },
        "axis_label": {
            "english": "Number of days",
            "dutch": "Aantal dagen"
        },
        "source": "https://www.knmi.nl/nederland-nu/klimatologie/maand-en-seizoensoverzichten/",
        "values": values
    })

climate_time_series

# Names of newborns
The SVB (Dutch social security bank) publishes yearly statistics on the number of names given to newborn babies. Only names occuring >25 times are included. The data is published in non-webscraper-friendly tables on their website, but those tables are fed by JSON textfiles which can be accessed as well.  

In [ ]:
import json
import requests

from collections import defaultdict

boy_names_urls = {
    "https://www.svbkindernamen.nl/nl/media/jongens-2016.txt": 2016,
    "https://www.svbkindernamen.nl/nl/media/jongens-2017.txt": 2017,
    "https://www.svbkindernamen.nl/nl/media/jongens-2018.txt": 2018,
    "https://www.svbkindernamen.nl/nl/media/jongens-2019.txt": 2019,
    "https://www.svbkindernamen.nl/nl/media/jongensnamen-2020-25-populair-json.txt": 2020,
}

girl_names_urls = {url.replace('jongens', 'meisjes'): year for url, year in boy_names_urls.items()}
urls = {**boy_names_urls, **girl_names_urls}

name_time_series = defaultdict(list)
for url, year in urls.items():
    names = json.loads(requests.get(url).content)['data']
    for record in names:
        name_time_series[record[0]].append(record[1])

name_time_series = dict(name_time_series)
name_time_series = [{
    "title": {
        "english": f"Number of newborns named {name}",
        "dutch": f"Aantal kinderen geboren met naam {name}"
    },
    "axis_label": {
        "english": "Number of newborns",
        "dutch": "Aantal kinderen"
    },
    "source": "hhttps://www.svbkindernamen.nl/",
    "values": values
} for name, values in name_time_series.items() if len(values) == len(boy_names_urls.keys())]

name_time_series

# Eredivisie
The Eredivisie (Dutch premier league soccer) produces a gold-mine of statistics. An easily accessible set of those can be found on ererat.nl. Applying pd.read_html() directly on the url doesn't work in this case, but it does after we use the good old requests + BeautifulSoup combo to isolate the table we need. 

Clubs can promote in and relegated out of the Eredivisie, so not all series are complete for every club. We only include series for clubs that were in the Eredevisie during the whole period we examine. 

In [ ]:
from bs4 import BeautifulSoup

urls = {
    "http://www.ererat.nl/asp/ererat_seizoen_clubstats.asp?report=5&sortitem=1&van=2014&tot=2020&clubid=-1": {"label": {"dutch": "Aantal rode kaarten voor voetbalclub", "english": "Number of red cards for soccer club"}, "axis_label": {"english": "Number of red cards", "dutch": "Aantal rode kaarten"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubstats.asp?van=2014&tot=2020&clubid=-1&report=4": {"label": {"dutch": "Aantal toeschouwers bij thuiswedstrijden voor voetbalclub", "english": "Number of spectators at homegames for soccer club"}, "axis_label": {"english": "Number of spectators", "dutch": "Aantal toeschouwers"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubgoals.asp?goals=1&van=2014&tot=2020&clubid=": {"label": {"dutch": "Aantal doelpunten voor voetbalclub", "english": "Number of goals for soccer club"}, "axis_label": {"english": "Number of goals", "dutch": "Aantal doelpunten"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubgoals.asp?goals=2&van=2014&tot=2020&clubid=-1": {"label": {"dutch": "Aantal doelpunten tegen voetbalclub", "english": "Number of goals against soccer club"}, "axis_label": {"english": "Number of goals", "dutch": "Aantal doelpunten"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubgoals.asp?goals=3&van=2014&tot=2020&clubid=-1": {"label": {"dutch": "Doelsaldo van voetbalclub", "english": "Goal difference for soccer club"}, "axis_label": {"english": "Goal difference", "dutch": "Doelsaldo"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubwedstrijden.asp?van=2014&tot=2020&clubid=-1&report=1": {"label": {"dutch": "Aantal wedstrijden geeindigd in winst voor voetbalclub", "english": "Number of games won by soccer club"}, "axis_label": {"english": "Number of games", "dutch": "Aantal wedstrijden"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubwedstrijden.asp?van=2014&tot=2020&clubid=-1&report=2": {"label": {"dutch": "Aantal wedstrijden geeindigd in gelijkspel voor voetbalclub", "english": "Number of games ended in a draw for soccer club"}, "axis_label": {"english": "Number of games", "dutch": "Aantal wedstrijden"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubwedstrijden.asp?van=2014&tot=2020&clubid=-1&report=3": {"label": {"dutch": "Aantal wedstrijden geeindigd in verlies voor voetbalclub", "english": "Number of games lost by soccer club"}, "axis_label": {"english": "Number of games", "dutch": "Aantal wedstrijden"}},
    "http://www.ererat.nl/asp/ererat_seizoen_clubstats.asp?van=2014&tot=2020&clubid=-1&report=3": {"label": {"dutch": "Aantal unieke spelers opgesteld door voetbalclub", "english": "Number of unique players positioned by soccer club"}, "axis_label": {"english": "Number of players", "dutch": "Aantal spelers"}},
}

replacements = {
    "2014/2015": 2014,
    "2015/2016": 2015,
    "2016/2017": 2016,
    "2017/2018": 2017,
    "2018/2019": 2018,
    "2019/2020": 2019,
    "2020/2021": 2020
}

eredivisie_time_series = []
for url, labels in urls.items():
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    table = soup.find_all("table", {"class": "data_table"})[0]
    
    df = pd.read_html(str(table))[0]
    df = df.iloc[1:]
    
    for season, year in replacements.items():
        df = df.replace(season, year)
    
    clubs = list(set(df[1]))
    
    for club in clubs:
        values = []
        for year in years:
            try:
                values.append(int(df[(df[1]==club) & (df[5]==year)][4]))
            except:
                continue
        
        if len(values) == 7:
            eredivisie_time_series.append({
                "title": {
                    "english": f"{labels['label']['english']} {club}",
                    "dutch": f"{labels['label']['dutch']} {club}"
                },
                "axis_label": {
                    "english": labels['axis_label']['english'],
                    "dutch": labels['axis_label']['dutch']
                },
                "source": url,
                "values": values
            })
                
eredivisie_time_series

# ProCyclingStats
As a cycling-enthusiast, I couldn't exclude statistics from the pro-cycling peloton from this project. Again, after isolating the table we want from a given webpage (containing the classification for an edition of a race), pd.read_html() parses that table into a DataFrame.

In [ ]:
races = {
    "Strade Bianche": "https://www.procyclingstats.com/race/strade-bianche/",
    "Milano-Sanremo": "https://www.procyclingstats.com/race/milano-sanremo/",
    "Ronde van Vlaanderen": "https://www.procyclingstats.com/race/ronde-van-vlaanderen/",
    "Liege-Bastogne-Liege": "https://www.procyclingstats.com/race/liege-bastogne-liege/",
    "Il Lombardia": "https://www.procyclingstats.com/race/il-lombardia/",
    "World Championship Individual Time Trial": "https://www.procyclingstats.com/race/world-championship-itt/",
    "World Championship Road Race": "https://www.procyclingstats.com/race/world-championship/",
    "Dutch National Championship Road Race": "https://www.procyclingstats.com/race/nc-netherlands/",
}

def remove_team_name_from_rider_name(rider_name, team_name):
    
    if not type(team_name) == float:
        return rider_name.replace(team_name, '')
    else:
        return rider_name

def switch_first_and_last_name(full_name):
    first_name = ' '.join([word for word in full_name.split(' ') if not word.isupper()])
    last_name = ' '.join([word for word in full_name.split(' ') if word.isupper()])
    
    return f"{first_name} {last_name.title()}"

from collections import defaultdict

classifications_per_rider = defaultdict(list)
pcs_time_series = []

for race, url in races.items():
    for year in years:
        result_url = url + str(year)         
        response = requests.get(result_url)
        soup = BeautifulSoup(response.content)

        table = soup.find("table", {"class": "basic results moblist10"})
        df = pd.read_html(str(table))[0]
        
        df['Rider'] = df.apply(lambda row: remove_team_name_from_rider_name(row['Rider'], row['Team']), axis=1)
        df['Rider'] = df.apply(lambda row: switch_first_and_last_name(row['Rider']), axis=1)
                
        for rider, result in dict(zip(df['Rider'], df['Rnk'])).items():
            try:
                classifications_per_rider[f"{rider} in {race}"].append(int(result))
            except:
                continue

classifications_per_rider

In [ ]:
pcs_time_series = [{
    "title": {
        "english": f"Classification of cyclist {label}",
        "dutch": f"Klassering van wielrenner {label}"
    },
    "axis_label": {
        "english": "Classification",
        "dutch": "Klassering"
    },
    "source": [url for race, url in races.items() if race in label][0],
    "values": values
} for label, values in classifications_per_rider.items() if len(values)==len(years)]

pcs_time_series

# CBS
The Dutch central statistics bureau - unsurprisingly - offers a huge amount of data. They even offer a Python package - `cbsodata` - that can be used to access it directly from Python, into a DataFrame. For now I selected two tables: one with crime statistics and one with outcomes of surveys related to happiness. 

In [ ]:
import pandas as pd
import cbsodata

toc = pd.DataFrame(cbsodata.get_table_list())

toc_yearly = toc[toc['Frequency'].isin(['Yearly', 'Perjaar'])]

toc_yearly_incl_2020 = toc_yearly[toc_yearly['Period'].str.contains('202')]
toc_yearly_incl_2020_ned = toc_yearly_incl_2020[toc_yearly_incl_2020['Identifier'].str.contains("NED")]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
toc_yearly_incl_2020_ned[['Title', 'Identifier']]

In [ ]:
identifier = "83648NED"

data = pd.DataFrame(cbsodata.get_data(identifier))

In [ ]:
years_str = [str(year) for year in years]

crime_types = [
    "17 Witwassen",
    "211 Vernieling aan auto",
    "23 Brandstichting / ontploffing",
    "52 Rijden onder invloed",
    "56 Joyriding",
    '61 Harddrugs',
    '62 Softdrugs',
]

replacements = {
    "17 Witwassen": "witwassen",
    "211 Vernieling aan auto": "vernieling aan auto",
    "23 Brandstichting / ontploffing": "brandstichting",
    "52 Rijden onder invloed": "rijden onder invloed",
    "56 Joyriding": "joyriding",
    '61 Harddrugs': "harddrugs",
    '62 Softdrugs': "softdrugs",
}

data_filtered = data[(data['SoortMisdrijf'].isin(crime_types)) & data['Perioden'].isin(years_str)]

for original, replacement in replacements.items():
    data_filtered['SoortMisdrijf'] = data_filtered['SoortMisdrijf'].replace(original, replacement)
    data_filtered['RegioS'] = data_filtered['RegioS'].str.replace(original, replacement)

data_filtered['SoortPerRegio'] = "Aantal misdrijven gerelateerd aan " + data_filtered['SoortMisdrijf'].str.lower() + " in " + data_filtered['RegioS']

data_filtered = data_filtered[~data_filtered['RegioS'].str.contains('Niet in te delen')]

In [ ]:
clean_up = {
    " (RE)": "",
    " (PV)": "",
    " (LD)": "",
    " (O.)": "",
    " Fryslân": "",
    " (gemeente)": ""
}

translations = {
    "witwassen": "money laundering",
    "vernieling aan auto": "destruction/vandalism of cars",
    "brandstichting": "arson",
    "rijden onder invloed": "drunk driving",
    "Aantal misdrijven gerelateerd aan": "Number of crimes related to",
}

def clean(text):
    for k, v in clean_up.items():
        text = text.replace(k, v)
    return text

def translate(text):
    for dutch, english in translations.items():
        text = text.replace(dutch, english)
    return text

crime_time_series = []

labels = list(set(data_filtered['SoortPerRegio']))

for label in labels:
    values = list(data_filtered[data_filtered['SoortPerRegio']==label].sort_values('Perioden')['TotaalGeregistreerdeMisdrijven_1'])
    
    assert len(values) == len(years)
    
    if not any([pd.isna(value) for value in values]) and all([value>100 for value in values]):
        label = clean(label)
        crime_time_series.append({
            "title": {
                "english": translate(label),
                "dutch": label
            },
            "axis_label": {
                "english": "Number of crimes",
                "dutch": "Aantal misdrijven"
            },
            "source": "https://opendata.cbs.nl/statline/#/CBS/nl/dataset/83648NED/table?fromstatweb",
            "values": values
        })

crime_time_series


In [ ]:
identifier = "84572NED"

data = pd.DataFrame(cbsodata.get_data(identifier))
data_happiness = data[(data['Marges'] == "Waarde") & data['Perioden'].isin(years_str)]
data_happiness['Label'] = f"Percentage mensen dat aangeeft gelukkig te zijn (" + data_happiness['KenmerkenPersonen'] + ")"

In [ ]:
translations = {
    "Percentage mensen dat aangeeft gelukkig te zijn": "Percentage of people indicating they're happy",
    "Geslacht: vrouwen": "Gender: female",
    "Geslacht: mannen": "Gender: male",
    "Leeftijd": "Age",
    "tot": "to",
    "jaar": "year",
    "of ouder": "or older",
    "Onderwijsniveau": "Education level",
    "Laag": "Low",
    "Middelbaar": "Middle",
    "Hoog": "High",
    "Migratieachtergrond": "Migration background",
    "westers": "Western",
    "niet": "not",
    "Totaal personen": "Total number of people",
    "Nederland": "the Netherlands"
}

def translate(text):
    for dutch, english in translations.items():
        text = text.replace(dutch, english)
    return text

happiness_time_series = []

labels = list(set(data_happiness['Label']))

for label in labels:
    values = list(data_happiness[data_happiness['Label']==label].sort_values('Perioden')['Gelukkig_1'])
    happiness_time_series.append({
            "title": {
                "english": translate(label),
                "dutch": label
            },
            "axis_label": {
                "english": "Number of people",
                "dutch": "Aantal mensen"
            },
            "source": "https://opendata.cbs.nl/statline/#/CBS/nl/dataset/84572NED/table?fromstatweb",
            "values": values
        })

happiness_time_series

cbs_time_series = happiness_time_series + crime_time_series


# Plotting
Now let's see how closely each series correlates to all others, showing plots of those that correlate strongly (positively or negatively), while they're from different categories. 


In [ ]:
import matplotlib.pyplot as plt

def plot(series1, series2, years, language):
    fig, ax = plt.subplots()
    
    label1 = series1['title'][language]
    label2 = series2['title'][language]
    y_label1 = series1['axis_label'][language]
    y_label2 = series2['axis_label'][language]
    plt.title(f"{label1}\nvs.\n{label2}")
    
    ax.plot(years, series1['values'], color="red", marker="o")
    ax.set_xlabel("Year",fontsize=14)
    ax.set_ylabel(y_label1,color="red")

    ax2 = ax.twinx()
    ax2.plot(years, series2['values'],color="blue",marker="o")
    ax2.set_ylabel(y_label2, color="blue")

    plt.xticks(years)

    plt.show()


In [ ]:
time_series = {
    "Cycling": pcs_time_series,
    "Eredivisie soccer": eredivisie_time_series,
    "Climate": climate_time_series,
    "Movies": imdb_time_series,
    "CBS": cbs_time_series
}
language = "english"

correlated_series = []
for category1, list1 in time_series.items():
    for series1 in list1:
        data1 = series1['values']
    
        for category2, list2 in time_series.items():
            for series2 in list2:
                if category1 == category2:
                    continue

                data2 = series2['values']        
                correlation = round(pd.Series(data1).corr(pd.Series(data2)), 4)

                if abs(correlation) > 0.95:
                    if [series2, series1, correlation] not in correlated_series:
                        correlated_series.append([series1, series2, correlation])
                        break

# Saving the dataset to JSON

In [ ]:
import json
import random

with open('all_data.json', 'w') as outfile:
    json.dump(time_series, outfile, indent=4)

random.shuffle(correlated_series)
print(correlated_series)
with open('correlated_series.json', 'w') as outfile:
    json.dump(correlated_series, outfile, indent=4)